<a href="https://colab.research.google.com/github/alanwuha/ce7455-nlp/blob/master/assignment-1/assignment-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download wikitext-2 dataset

In [1]:
!rm -r data
!mkdir data
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip -P /content/data
!unzip /content/data/wikitext-2-v1.zip -d /content/data

rm: cannot remove 'data': No such file or directory
--2020-02-06 04:57:06--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.106.158
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.106.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4475746 (4.3M) [application/zip]
Saving to: ‘/content/data/wikitext-2-v1.zip’

wikitext-2-v1.zip   100%[===================>]   4.27M  8.15MB/s    in 0.5s    

2020-02-06 04:57:07 (8.15 MB/s) - ‘/content/data/wikitext-2-v1.zip’ saved [4475746/4475746]

Archive:  /content/data/wikitext-2-v1.zip
   creating: /content/data/wikitext-2/
  inflating: /content/data/wikitext-2/wiki.test.tokens  
  inflating: /content/data/wikitext-2/wiki.valid.tokens  
  inflating: /content/data/wikitext-2/wiki.train.tokens  


### Specify arg variables

In [0]:
data = './data/wikitext-2'  # location of the data corpus
model = 'LSTM'              # type of recurrent net
emsize = 200                # size of word embeddings
nhid = 200                  # number of hidden units per layer
nlayers = 2                 # number of layers
lr = 20                     # initial learning rate
clip = 0.25                 # gradient clipping
epochs = 40                 # upper epoch limit
batch_size = 20             # batch size
bptt = 35                   # sequence length
dropout = 0.2               # dropout applied to layers (0 = no dropout)
tied = True                 # tie the word embedding and softmax weights
seed = 1111                 # random seed
cuda = True                # use CUDA
log_interval = 200          # report interval
save = 'model.pt'           # path to save the final model
onnx_export = ''            # path to export the final model in onnx format
nhead = 2                   # the number of heads in the encoder/decoder of the transformer model

### data.py

In [0]:
import os
from io import open
import torch

class Dictionary(object):
  def __init__(self):
    self.word2idx = {}
    self.idx2word = []

  def add_word(self, word):
    if word not in self.word2idx:
      self.idx2word.append(word)
      self.word2idx[word] = len(self.idx2word) - 1
    return self.word2idx[word]

  def __len__(self):
    return len(self.idx2word)

class Corpus(object):
  def __init__(self, path):
    self.dictionary = Dictionary()
    self.train = self.tokenize(os.path.join(path, 'wiki.train.tokens'))
    self.valid = self.tokenize(os.path.join(path, 'wiki.valid.tokens'))
    self.test = self.tokenize(os.path.join(path, 'wiki.test.tokens'))

  def tokenize(self, path):
    """ Tokenizes a text file. """
    assert os.path.exists(path)

    # Add words to the dictionary
    with open(path, 'r', encoding='utf8') as f:
      for line in f:
        words = line.split() + ['<eos>']
        for word in words:
          self.dictionary.add_word(word)

    # Tokenize file content
    with open(path, 'r', encoding='utf8') as f:
      idss = []
      for line in f:
        words = line.split() + ['<eos>']
        ids = []
        for word in words:
          ids.append(self.dictionary.word2idx[word])
        idss.append(torch.tensor(ids).type(torch.int64))
      ids = torch.cat(idss)

    return ids

### main.py

In [0]:
import time
import math
import os
import torch
import torch.nn as nn

# Set the random seed manually for reproducibility.
torch.manual_seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

corpus = Corpus(data)

def batchify(data, bsz):
  # Work out how cleanly we can divide the dataset into bsz parts.
  nbatch = data.size(0) // bsz
  # Trim off any extra elements that wouldn't cleanly fit (remainders).
  data = data.narrow(0, 0, nbatch * bsz)
  # Evenly divide the data across the bsz batches.
  data = data.view(bsz, -1).t().contiguous()
  return data.to(device)

eval_batch_size = 10
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

ntokens = len(corpus.dictionary)
# model
criterion = nn.CrossEntropyLoss()



In [18]:
corpus.train.size()

torch.Size([2088628])

In [0]:
train_data = batchify(corpus.train, batch_size)